In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from zipfile import ZipFile
import requests
import io


In [ ]:
# Loading files for September 2020 amd September 2021

citi2020 = pd.read_csv('https://s3.amazonaws.com/tripdata/202009-citibike-tripdata.csv.zip')


Opening 2021 file. This method is used to unpack the zip because it has more than a single csv.


In [ ]:
# Here we are getting the zip file
citibikeZipFile = requests.get('https://s3.amazonaws.com/tripdata/202109-citibike-tripdata.csv.zip')

# Here we are uncompressing it and reading the whole package into a variable
zipFiles = ZipFile(io.BytesIO(citibikeZipFile.content))

# And here we are looping through the package and printing the names of the files
for name in zipFiles.namelist():
  print(name)

# Finally, we can take that loaded uncopressed package and use it to just load one file to the Pandas dataframe:
citi2021 = pd.read_csv(zipFiles.open('202109-citibike-tripdata.csv'))

Calculating the total number of trips for the two periods

In [ ]:
#2020
#First we count the number of trip per station and sum the results. 

citibikeStations = citi2020.groupby(['start station id']).agg({'tripduration':'count'}).reset_index()
#Changing the tripduration column into tripcount to reflect the trips taken
citibikeStations.rename(columns={'tripduration':'tripCount'},inplace=True)
#Total number of all trips
print("There were", citibikeStations.tripCount.sum(),"trips in September 2020.")

In [ ]:
#2021
#First we count the number of trip per station and sum the results. 

citibikes21 = citi2021.groupby(['start_station_id']).agg({'started_at':'count'}).reset_index()
#Changing the tripduration column into tripcount to reflect the trips taken
citibikes21.rename(columns={'started_at':'tripCount'},inplace=True)
#Total number of all trips
print("There were", citibikes21.tripCount.sum(),"trips in September 2021.")

Top 5 stations by number of trip starts


In [ ]:
#2020
citi2020['start station name'].value_counts().head(5)


In [ ]:
#2021
citi2021['start_station_name'].value_counts().head(5)

Top 5 stations by number of trip stops


In [ ]:
#2020
citi2020.groupby(['start station name']).agg({ 'stoptime':'count'}).sort_values(by='stoptime',ascending=False).reset_index().head()


In [ ]:
#2021
citi2021.groupby(['start_station_name']).agg({ 'ended_at':'count'}).sort_values(by='ended_at',ascending=False).reset_index().head()


Median number of starts

In [ ]:
#2020
citi2020.groupby(['start station name']).agg({ 'starttime':'count'}).median()

In [ ]:
#2021
citi2021.groupby(['start_station_name']).agg({ 'started_at':'count'}).median()

Median number of stops

In [ ]:
#2021
citi2020.groupby(['end station name']).agg({ 'stoptime':'count'}).median()

In [ ]:
#2021

citi2021.groupby(['end_station_name']).agg({ 'ended_at':'count'}).median()

In [ ]:
citi2020.head()

In [ ]:
citi2021.columns

In [ ]:
#Starts Geo 2020
citiStartsGeo20 = citi2020.groupby(['start station name']).agg({'start station latitude':'min','start station longitude':'min', 'tripduration':'count'}).reset_index()

In [ ]:
citiStartsGeo20.rename(columns={'tripduration':'tripCount'},inplace=True)

In [ ]:
#Stops Geo 2020

citiStopsGeo20 = citi2020.groupby(['end station name']).agg({'end station latitude':'min','end station longitude':'min', 'tripduration':'count'}).reset_index()

In [ ]:
citiStopsGeo20.rename(columns={'tripduration':'tripCount'},inplace=True)

In [ ]:
#Starts Geo 2021
citiStartsGeo21 = citi2021.groupby(['start_station_name']).agg({'start_lat':'min','start_lng':'min', 'start_station_id':'count'}).reset_index()

In [ ]:
citiStartsGeo20.rename(columns={'start_station_id':'tripCount'},inplace=True)

In [ ]:
#Stops Geo 2021
citiStopsGeo21 = citi2021.groupby(['end_station_name']).agg({'end_lat':'min','end_lng':'min', 'end_station_id':'count'}).reset_index()

In [ ]:
citiStopsGeo21.rename(columns={'end_station_id':'tripCount'},inplace=True)

In [ ]:
#Dropping unneccesary columns

# citi2020.drop(columns={'tripduration', 'starttime', 'stoptime', 'bikeid', 'usertype', 'birth year', 'gender','end station id'})

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
#Starts Geo 2020
citiStartsGeo20 = gpd.GeoDataFrame(data=citiStartsGeo20, geometry=gpd.points_from_xy(x=citiStartsGeo20['start station longitude'], y=citiStartsGeo20['start station latitude']), crs='epsg:4326')

In [ ]:
#Stops Geo 2020
citiStopsGeo20 = gpd.GeoDataFrame(data=citiStopsGeo20, geometry=gpd.points_from_xy(x=citiStopsGeo20['end station longitude'], y=citiStopsGeo20['end station latitude']), crs='epsg:4326')

In [ ]:
#Starts Geo 2021

citiGeoStarts21 = gpd.GeoDataFrame(data=citiStartsGeo21, geometry=gpd.points_from_xy(x=citiStartsGeo21['start_lng'], y=citiStartsGeo21['start_lat']), crs='epsg:4326')


In [ ]:
citiStopsGeo21.columns

In [ ]:
#Stops Geo 2021
citiGeoStops21 = gpd.GeoDataFrame(data=citiStopsGeo21, geometry=gpd.points_from_xy(x=citiStopsGeo21['end_lng'], y=citiStopsGeo21['end_lat']), crs='epsg:4326')


In [ ]:
#Starts Geo 2020
citiStartsGeo20.to_file("citiStartsGeo20.geojson", driver='GeoJSON')


In [ ]:
citiStartsGeo20.to_file("citiStartsGeo20.geojson", driver='GeoJSON')
citiStopsGeo20.to_file("citiStopsGeo20.geojson", driver='GeoJSON')
citiGeoStarts21.to_file("citiGeoStarts21.geojson", driver='GeoJSON')
citiGeoStops21.to_file("citiGeoStops21.geojson", driver='GeoJSON')







In [ ]:
citiGeo = gpd.GeoDataFrame(data=citi2020, geometry=gpd.points_from_xy(x=citi2020['start_lng'], y=citi2020['start station latitude']), crs='epsg:4326')